In [38]:
import cv2
from cv2 import aruco
from datetime import datetime
from loguru import logger
from pathlib import Path
from typing import Optional, cast, Final
import awkward as ak
from cv2.typing import MatLike
import numpy as np
from matplotlib import pyplot as plt
import awkward as ak
from awkward import Record as AwkwardRecord, Array as AwkwardArray

In [39]:
NDArray = np.ndarray
OBJECT_POINTS_PARQUET = Path("output") / "object_points.parquet"
DICTIONARY: Final[int] = aruco.DICT_4X4_50
# 400mm
MARKER_LENGTH: Final[float] = 0.4

A_CALIBRATION_PARQUET = Path("output") / "a-ae_08.parquet"
B_CALIBRATION_PARQUET = Path("output") / "b-ae_09.parquet"
C_CALIBRATION_PARQUET = Path("output") / "c-af_03.parquet"

In [40]:
aruco_dict = aruco.getPredefinedDictionary(DICTIONARY)
def read_camera_calibration(path: Path) -> tuple[MatLike, MatLike]:
    cal = ak.from_parquet(path)[0]
    camera_matrix = cast(MatLike, ak.to_numpy(cal["camera_matrix"]))
    distortion_coefficients = cast(MatLike, ak.to_numpy(cal["distortion_coefficients"]))
    return camera_matrix, distortion_coefficients

ops = ak.from_parquet(OBJECT_POINTS_PARQUET)
detector = aruco.ArucoDetector(
    dictionary=aruco_dict, detectorParams=aruco.DetectorParameters()
)

total_ids = cast(NDArray, ak.to_numpy(ops["ids"])).flatten()
total_corners = cast(NDArray, ak.to_numpy(ops["corners"])).reshape(-1, 4, 3)
ops_map: dict[int, NDArray] = dict(zip(total_ids, total_corners))
# display("ops_map", ops_map)

In [41]:
def process(
    frame: MatLike,
    cam_mtx: MatLike,
    dist_coeffs: MatLike,
    target: Optional[MatLike] = None,
) -> tuple[MatLike, Optional[MatLike], Optional[MatLike]]:
    if target is None:
        target = frame.copy()
    grey = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    # pylint: disable-next=unpacking-non-sequence
    markers, ids, rejected = detector.detectMarkers(grey)
    # `markers` is [N, 1, 4, 2]
    # `ids` is [N, 1]
    ret_rvec: Optional[MatLike] = None
    ret_tvec: Optional[MatLike] = None
    if ids is not None:
        markers = np.reshape(markers, (-1, 4, 2))
        ids = np.reshape(ids, (-1, 1))
        # logger.info("markers={}, ids={}", np.array(markers).shape, np.array(ids).shape)
        ips_map: dict[int, NDArray] = {}
        for cs, id in zip(markers, ids):
            id = int(id)
            cs = cast(NDArray, cs)
            ips_map[id] = cs
            center = np.mean(cs, axis=0).astype(int)
            GREY = (128, 128, 128)
            # logger.info("id={}, center={}", id, center)
            cv2.circle(target, tuple(center), 5, GREY, -1)
            cv2.putText(
                target,
                str(id),
                tuple(center),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                GREY,
                2,
            )
            # BGR
            RED = (0, 0, 255)
            GREEN = (0, 255, 0)
            BLUE = (255, 0, 0)
            YELLOW = (0, 255, 255)
            color_map = [RED, GREEN, BLUE, YELLOW]
            for color, corners in zip(color_map, cs):
                corners = corners.astype(int)
                target = cv2.circle(target, corners, 5, color, -1)
        # https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga50620f0e26e02caa2e9adc07b5fbf24e
        ops: NDArray = np.empty((0, 3), dtype=np.float32)
        ips: NDArray = np.empty((0, 2), dtype=np.float32)
        for id, ip in ips_map.items():
            try:
                op = ops_map[id]
                assert ip.shape == (4, 2), f"corners.shape={ip.shape}"
                assert op.shape == (4, 3), f"op.shape={op.shape}"
                ops = np.concatenate((ops, op), axis=0)
                ips = np.concatenate((ips, ip), axis=0)
            except KeyError:
                logger.warning("No object points for id={}", id)
                continue
        assert len(ops) == len(ips), f"len(ops)={len(ops)} != len(ips)={len(ips)}"
        if len(ops) > 0:
            # https://docs.opencv.org/4.x/d5/d1f/calib3d_solvePnP.html
            # https://docs.opencv.org/4.x/d5/d1f/calib3d_solvePnP.html#calib3d_solvePnP_flags
            ret, rvec, tvec = cv2.solvePnP(
                objectPoints=ops,
                imagePoints=ips,
                cameraMatrix=cam_mtx,
                distCoeffs=dist_coeffs,
                flags=cv2.SOLVEPNP_SQPNP,
            )
            # ret, rvec, tvec, inliners = cv2.solvePnPRansac(
            #     objectPoints=ops,
            #     imagePoints=ips,
            #     cameraMatrix=camera_matrix,
            #     distCoeffs=distortion_coefficients,
            #     flags=cv2.SOLVEPNP_SQPNP,
            # )
            if ret:
                cv2.drawFrameAxes(
                    target,
                    cam_mtx,
                    dist_coeffs,
                    rvec,
                    tvec,
                    MARKER_LENGTH,
                )
                ret_rvec = rvec
                ret_tvec = tvec
    return target, ret_rvec, ret_tvec

In [42]:
A_IMG = Path("dumped/batch_three/video-20241224-154256-a.png")
B_IMG = Path("dumped/batch_three/video-20241224-154302-b.png")
C_IMG = Path("dumped/batch_three/video-20241224-154252-c.png")
C_PRIME_IMG = Path("dumped/batch_three/video-20241224-153926-c-prime.png")

a_img = cv2.imread(str(A_IMG))
b_img = cv2.imread(str(B_IMG))
c_img = cv2.imread(str(C_IMG))
c_prime_img = cv2.imread(str(C_PRIME_IMG))

a_mtx, a_dist = read_camera_calibration(A_CALIBRATION_PARQUET)
b_mtx, b_dist = read_camera_calibration(B_CALIBRATION_PARQUET)
c_mtx, c_dist = read_camera_calibration(C_CALIBRATION_PARQUET)

In [43]:
a_result_img, a_rvec, a_tvec = process(a_img, a_mtx, a_dist)
# plt.imshow(cv2.cvtColor(a_result_img, cv2.COLOR_BGR2RGB))

/var/folders/cj/0zmvpygn7m72m42lh6x_hcgw0000gn/T/ipykernel_79393/542219436.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  id = int(id)


In [44]:
b_result_img, b_rvec, b_tvec = process(b_img, b_mtx, b_dist)
# plt.imshow(cv2.cvtColor(b_result_img, cv2.COLOR_BGR2RGB))

/var/folders/cj/0zmvpygn7m72m42lh6x_hcgw0000gn/T/ipykernel_79393/542219436.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  id = int(id)


In [45]:
c_result_img, c_rvec, c_tvec = process(c_img, c_mtx, c_dist)
c_prime_result_img, c_prime_rvec, c_prime_tvec = process(c_prime_img, c_mtx, c_dist)

/var/folders/cj/0zmvpygn7m72m42lh6x_hcgw0000gn/T/ipykernel_79393/542219436.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  id = int(id)


In [46]:
params = AwkwardArray(
    [
        {
            "name": "a-ae_08",
            "rvec": a_rvec,
            "tvec": a_tvec,
            "camera_matrix": a_mtx,
            "distortion_coefficients": a_dist,
        },
        {
            "name": "b-ae_09",
            "rvec": b_rvec,
            "tvec": b_tvec,
            "camera_matrix": b_mtx,
            "distortion_coefficients": b_dist,
        },
        {
            "name": "c-af_03",
            "rvec": c_rvec,
            "tvec": c_tvec,
            "camera_matrix": c_mtx,
            "distortion_coefficients": c_dist
        },
        {
            "name": "c-prime-af_03",
            "rvec": c_prime_rvec,
            "tvec": c_prime_tvec,
            "camera_matrix": c_mtx,
            "distortion_coefficients": c_dist
        }
    ]
)
display("params", params)
ak.to_parquet(params, Path("output") / "params.parquet")

'params'

<Array [{name: 'a-ae_08', rvec: ..., ...}, ...] type='4 * {name: string, rv...'>

  created_by: parquet-cpp-arrow version 14.0.1
  num_columns: 5
  num_rows: 4
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

In [47]:
cv2.imwrite("output/a_result_img.png", a_result_img)
cv2.imwrite("output/b_result_img.png", b_result_img)
cv2.imwrite("output/c_result_img.png", c_result_img)
cv2.imwrite("output/c_prime_result_img.png", c_prime_result_img)

True